In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import cv2 
import numpy as np 

In [2]:
# os.listdir("./finished/train/dataraw/hires") #listing the contents in the location 

In [3]:
file_path="./finished/train/dataraw/hires/" #path

In [4]:
def create_lr_data(img_path="",img_file="",compression_factor=10,w=0,h=0):
  
  img=cv2.imread(img_path+img_file)
  if w!=0 and h!=0:
    newimg1=cv2.resize(img,(w,h),interpolation=cv2.INTER_CUBIC)
    # cv2_imshow(newimg1)
    return newimg1
  else:
    w=img.shape[1]  
    h=img.shape[0]  
    newimg1=cv2.resize(img,(int(w/compression_factor),int(h/compression_factor)),interpolation=cv2.INTER_CUBIC)
    # cv2_imshow(newimg1)
    return newimg1



In [5]:
hr_arr=[]
lr_arr=[]
error_images=[]
for imgs in os.listdir(file_path):
  try:
    img_hr=create_lr_data(img_path=file_path, img_file=imgs, w=800,h=800)
    hr_arr.append(img_hr)
    img_lr=create_lr_data(img_path=file_path, img_file=imgs, w=200,h=200)
    lr_arr.append(img_lr)
  except Exception as e:
    print(e)
    error_images.append(imgs)
  # cv2_imshow(img_lr)
  # break
hr_arr=np.array(hr_arr)
lr_arr=np.array(lr_arr)

In [6]:
import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import UpSampling2D
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers import Conv2DTranspose
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

#Model

In [7]:
###MODEL

model = keras.Sequential()

inp= tf.keras.Input(shape=(200, 200, 3))
#1st convolution
conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_1')(inp)

#2nd convolution
conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_2')(conv1)

#3rd convolution
conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_3')(conv2)

#skip connection between the 1st and 3rd convolution
skip = tf.keras.layers.Add()([conv1,conv3])


#bottleneck
bot = tf.keras.layers.Conv2D(64, (1,1), activation='relu')(skip)

#upsampling layer

up= tf.keras.layers.Conv2DTranspose(64, (2,2),strides=(4,4), padding='same', input_shape=(200, 200, 64))(bot)
#bottleneck
x = tf.keras.layers.Conv2D(4, (1,1), activation='relu')(up)



#RECONSTUCTION STAGE ONE

#first block

#Asymmetric convolution 1
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)

#Asymmetric convolution 2
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(y)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)

#Asymmetric convolution 3
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(y)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)
                           
#Asymmetric convolution 4
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(y)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)

#Residual connection
x = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)  # Residual Connection

x = tf.keras.layers.Add()([x, y])  # Addition

#second block

#Asymmetric convolution 1
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)

#Asymmetric convolution 2
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(y)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)

#Asymmetric convolution 3
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(y)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)
                           
#Asymmetric convolution 4
y = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(y)

y = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(y)

#Residual connection
x = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)  # Residual Connection

x = tf.keras.layers.Add()([x, y])  # Addition


#RECONSTUCTION STAGE TWO
#INCEPTION BLOCK OF 5 TOWERS
                           
T11 = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)

T12 = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(T11)
                           
T13 = tf.keras.layers.Conv2D(16, (5, 1), padding='same')(T12)

T1 = tf.keras.layers.Conv2D(16, (1, 5), activation='relu', padding='same')(T13)
                           
                           
T21 = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)

T22 = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(T21)
                           
T23 = tf.keras.layers.Conv2D(16, (5, 1), padding='same')(T22)

T2 = tf.keras.layers.Conv2D(16, (1, 5), activation='relu', padding='same')(T23)
                           

T31 = tf.keras.layers.Conv2D(16, (3, 1), padding='same')(x)

T32 = tf.keras.layers.Conv2D(16, (1, 3), activation='relu', padding='same')(T31)
                           
T33 = tf.keras.layers.Conv2D(16, (5, 1), padding='same')(T32)

T3 = tf.keras.layers.Conv2D(16, (1, 5), activation='relu', padding='same')(T33)  
                           
                           
T41 = tf.keras.layers.Conv2D(16, (7, 1), padding='same')(x)

T42 = tf.keras.layers.Conv2D(16, (1, 7), activation='relu', padding='same')(T41)
                           
T43 = tf.keras.layers.Conv2D(16, (9, 1), padding='same')(T42)

T4 = tf.keras.layers.Conv2D(16, (1, 9), activation='relu', padding='same')(T43)  
                           
                           
T51 = tf.keras.layers.Conv2D(16, (7, 1), padding='same')(x)

T52 = tf.keras.layers.Conv2D(16, (1, 7), activation='relu', padding='same')(T51)
                           
T53 = tf.keras.layers.Conv2D(16, (9, 1), padding='same')(T52)

T5 = tf.keras.layers.Conv2D(16, (1, 9), activation='relu', padding='same')(T53)  
                           
                           


#CONCATENATION OF OUTPUT OF FIVE TOWERS
                 
T= tf.keras.layers.Concatenate(axis=3)([T1,T2,T3,T4,T5])


#BOTTLENECK
y_pred = tf.keras.layers.Conv2D(3, (1,1), activation='relu',)(T)

model = tf.keras.Model(inp,y_pred)

## **Model Summary**

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 200, 200, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 200, 200, 64) 36928       conv_1[0][0]                     
__________________________________________________________________________________________________
conv_3 (Conv2D)                 (None, 200, 200, 64) 36928       conv_2[0][0]                     
______________________________________________________________________________________________

In [9]:
Adam1 = keras.optimizers.Adam(learning_rate=0.0003)

In [10]:
model.compile(loss ='mean_squared_error', optimizer=Adam1, metrics=['accuracy'])

# **Training using Tensorboard**

In [11]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [12]:
import tensorboard
tensorboard.__version__

'2.4.1'

In [13]:
log_dir = "logs_150/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [14]:
my_callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)]

In [15]:
model.fit(lr_arr, hr_arr, epochs=200, batch_size=2, callbacks=my_callbacks)

Train on 4128 samples


/home/cpslab/ml2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Epoch 1/200
4128/4128 [==============================] - 760s 184ms/sample - loss: 486.0151 - acc: 0.7402
Epoch 2/200
4128/4128 [==============================] - 692s 168ms/sample - loss: 254.3669 - acc: 0.8490
Epoch 3/200
4128/4128 [==============================] - 692s 168ms/sample - loss: 243.3304 - acc: 0.8586
Epoch 4/200
4128/4128 [==============================] - 693s 168ms/sample - loss: 235.8107 - acc: 0.8733
Epoch 5/200
4128/4128 [==============================] - 689s 167ms/sample - loss: 233.6769 - acc: 0.8749
Epoch 6/200
4128/4128 [==============================] - 687s 166ms/sample - loss: 229.2638 - acc: 0.8784
Epoch 7/200
4128/4128 [==============================] - 687s 166ms/sample - loss: 226.1828 - acc: 0.8791
Epoch 8/200
4128/4128 [==============================] - 687s 166ms/sample - loss: 225.9040 - acc: 0.8787
Epoch 9/200
4128/4128 [==============================] - 687s 166ms/sample - loss: 223.7651 - acc: 0.8810
Epoch 10/200
4128/4128 [======================

In [16]:
!mkdir -p saved_model
model.save('saved_model/my_model_200_')

In [17]:
!mkdir -p saved_model_h5
model.save('my_h5_model_200_.h5')

In [18]:
%tensorboard --logdir logs

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

## Preprocessed and processed image outputs

In [ ]:
img_lr.shape

In [ ]:
cv2.imshow('image',img_lr.reshape(200,200,3))

In [ ]:
cv2.imshow('image',img_hr)

In [ ]:
img_lr=img_lr.reshape((1,)+img_lr.shape)

In [ ]:
img_lr.shape

In [ ]:
out=model.predict([img_lr])

# **SSIM** **CALCULATION**

In [ ]:
im1 = tf.image.convert_image_dtype(img_hr, tf.float32)
im2 = tf.image.convert_image_dtype(out, tf.float32)

In [ ]:
ssim2 = tf.image.ssim(im1, im2, max_val=1.0, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03)

In [ ]:
print(ssim2)

# **OUTPUT IMAGE**

In [ ]:
cv2_imshow(out.reshape(800,800,3))

In [ ]:
model.history

In [ ]:
dot_img_file = './drive/My Drive/model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, dpi=300,show_shapes=True)

In [ ]:
!tensorboard --logdir=logs --host localhost --port 8088